<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Source" data-toc-modified-id="Data-Source-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Source</a></span></li><li><span><a href="#Imports-and-Update-The-Latest-Data" data-toc-modified-id="Imports-and-Update-The-Latest-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports and Update The Latest Data</a></span></li><li><span><a href="#Historical-air-4-Thai-Data" data-toc-modified-id="Historical-air-4-Thai-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Historical air 4 Thai Data</a></span></li><li><span><a href="#Power-Plants" data-toc-modified-id="Power-Plants-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Power Plants</a></span></li><li><span><a href="#Weather-Files" data-toc-modified-id="Weather-Files-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Weather Files</a></span><ul class="toc-item"><li><span><a href="#Assemble-from-raw-data" data-toc-modified-id="Assemble-from-raw-data-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Assemble from raw data</a></span></li><li><span><a href="#Update-weather-all-cities" data-toc-modified-id="Update-weather-all-cities-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Update weather all cities</a></span></li></ul></li><li><span><a href="#Holiday-In-Thailand" data-toc-modified-id="Holiday-In-Thailand-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Holiday In Thailand</a></span></li><li><span><a href="#Hotspots-Data" data-toc-modified-id="Hotspots-Data-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Hotspots Data</a></span></li></ul></div>

# Data Source 
1. Berekely Earth 'http://berkeleyearth.lbl.gov/air-quality/maps/cities/Thailand/'
2. Screaped Air4Thai Data 'http://air4thai.pcd.go.th/webV2/history/'
3. CDC Data 'https://www.cmuccdc.org/download_json/'
4. Old Air4Thai data from Thailand EPA

# Imports and Update The Latest Data

In [1]:
#  Load the "autoreload" extension so that code can change
%load_ext autoreload
%reload_ext autoreload
from pathlib import Path

#  always reload modules so that as you change code in src, it gets loaded
%autoreload 2
%matplotlib inline

import sys
sys.path.append('../')
from src.imports import *
from src.data.download_data import *
from src.data.read_data import *
from src.data.fire_data import *
from src.data.weather_data import *
from src.gen_functions import *

In [2]:
b_folder='../data/pm25/'
a4th_folder='../data/air4thai_hourly/'
cdc_folder = '../data/cdc_data/'
aqm_folder = '../data/aqm_hourly2/'
w_folder = '../data/weather_cities/'

In [3]:
# Data from Berkeley Earth Projects: 
download_b_data(data_folder='../data/pm25/', url='http://berkeleyearth.lbl.gov/air-quality/maps/cities/Thailand/')
get_city_info(data_folder='../data/pm25/')

100% [............................................................................] 805931 / 805931

In [4]:
download_cdc_data(station_url='https://www.cmuccdc.org/api/ccdc/stations', 
                  dl_url= 'https://www.cmuccdc.org/download_json/', 
                  data_folder='../data/cdc_data/')

  0%|                                                                                                             | 0/404 [00:00<?, ?it/s]

number of stations 407


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 404/404 [17:08<00:00,  2.55s/it]


In [252]:
update_last_air4Thai(url='http://air4thai.pcd.go.th/webV2/history/',data_folder='../data/air4thai_hourly/')



0it [00:00, ?it/s]

['02t', '03t', '05t', '08t', '10t', '11t', '12t', '13t', '14t', '16t', '17t', '18t', '19t', '20t', '21t', '22t', '24t', '25t', '26t', '27t', '28t', '29t', '30t', '31t', '32t', '33t', '34t', '35t', '36t', '37t', '38t', '39t', '40t', '41t', '42t', '43t', '44t', '46t', '47t', '50t', '52t', '53t', '54t', '57t', '58t', '59t', '60t', '61t', '62t', '63t', '67t', '68t', '69t', '70t', '71t', '72t', '73t', '74t', '75t', '76t', '77t', '79t', '80t', '81t', '82t', '83t', '84t', 'm1', 'm4', 'm8', 'm9', 'o10', 'o20', 'o22', 'o23', 'o24', 'o25', 'o26', 'o27', 'o28', 'o29']




1it [00:24, 24.91s/it]

2it [00:50, 25.13s/it]

3it [01:16, 25.27s/it]

4it [01:42, 25.47s/it]

5it [02:08, 25.75s/it]

6it [02:34, 25.95s/it]

7it [03:00, 25.90s/it]

8it [03:26, 26.01s/it]

9it [03:53, 26.06s/it]

10it [04:19, 26.16s/it]

11it [04:45, 26.18s/it]

12it [05:12, 26.27s/it]

13it [05:38, 26.28s/it]

14it [06:04, 26.31s/it]

15it [06:31, 26.31s/it]

16it [06:57, 26.35s/it]

17it [07:24, 26.35s/it]

18it [07:50, 26.34s/it]

19it [08:16, 26.37s/it]

20it [08:43, 26.34s/it]

21it [09:09, 26.34s/it]

22it [09:35, 26.35s/it]

23it [10:02, 26.40s/it]

24it [10:24, 25.14s/it]

25it [10:50, 25.28s/it]

26it [11:16, 25.64s/it]

27it [11:42, 25.83s/it]

28it [12:09, 26.05s/it]

29it [12:35, 26.15s/it]

30it [13:02, 26.29s/it]

31it [13:29, 26.38s/it]

32it [13:55, 26.39s/it]

33it [14:21, 26.37s/it]

34it [14:47, 26.32s/it]

35it [15:14, 26.32s/it]

36it [15:40, 26.33s/it]

37it [16:07, 26.37s/it]

38it [16:33, 26.33s/it]

39it [16:59, 26.38s/it]

40it [17:26, 26.39s/it]

41it [1

# Historical air 4 Thai Data

In [99]:
# load stations information
station_info_file = aqm_folder + 'stations_locations.json'
with open(station_info_file, 'r',encoding="utf8") as f:
    station_info = json.load(f)
station_info = station_info['stations']

# find stations in Chiangmai and parase that files
cm_station_ids = []
for i, stations in enumerate(station_info):
    if 'Chiang Mai' in stations['areaEN']:
        # ignore station that start with o
        if 'o' not in stations['stationID']:
            cm_station_ids.append(stations['stationID'])
            print(i)
print(cm_station_ids)

123
124
125
['35t', '36t', 'm9']


In [22]:
# parase historical data 
for station_id in cm_station_ids:
    # find all files that start with this stations
    p = Path(aqm_folder)
    filenames = []
    for i in p.glob('**/*.xlsx'):
        if station_id in i.name:
            filenames.append(str(i))
        
    # if filename exist load that file
    if len(filenames) >0:

        save_filename = aqm_folder + 'process/' + station_id + '.csv'
        print('save filename', save_filename)
        station_data = read_his_xl(filenames[0])
        #print(station_data.head())

        # save the data if the dataframe is not empty
        if len(station_data)> 0:
            station_data.to_csv(save_filename,index=True)

save filename ../data/aqm_hourly2/process/35t.csv
save filename ../data/aqm_hourly2/process/36t.csv


In [45]:
cm_data1 = pd.read_csv(aqm_folder + 'process/35t.csv')
cm_data1['datetime'] = pd.to_datetime(cm_data1['datetime'])
cm_data1 = cm_data1.set_index('datetime')
# keep only gas columns
cm_data1 = cm_data1[['CO', 'O3', 'NO2', 'SO2', 'PM10', 'PM2.5']]

cm_data2 = pd.read_csv(aqm_folder + 'process/36t.csv')
cm_data2['datetime'] = pd.to_datetime(cm_data2['datetime'])
cm_data2 = cm_data2.set_index('datetime')


# Power Plants

In [92]:
url = 'https://th.wikipedia.org/wiki/%E0%B8%A3%E0%B8%B2%E0%B8%A2%E0%B8%8A%E0%B8%B7%E0%B9%88%E0%B8%AD%E0%B9%82%E0%B8%A3%E0%B8%87%E0%B9%84%E0%B8%9F%E0%B8%9F%E0%B9%89%E0%B8%B2%E0%B9%83%E0%B8%99%E0%B9%84%E0%B8%97%E0%B8%A2'
url = 'https://en.wikipedia.org/wiki/List_of_power_stations_in_Thailand'
table_list = pd.read_html(url)
len(table_list)
p_folder = '../data/power_plants/'
for i, table in enumerate(table_list):
    table.to_csv(p_folder + f'table_eng{i}.csv',index=False)

# Weather Files

In [55]:
folders = glob('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities/*/')
len(folders)

9

In [92]:
for folder in folders:
    city_name = Path(folder).name
    parent_folder = Path(folder).parent
    w_files = glob(folder + '/*.csv')
    print(city_name, len(w_files))
    filename = str(Path(folders[0]).parent) + '/' + city_name + '.csv'
    
    # concatenate all files 
    df_all = pd.DataFrame()
    for file in w_files:
        try:
            df = pd.read_csv(file)
            
        except:
            pass
        else:
            df_all = pd.concat([df_all, df], ignore_index=True)
    
    # drop missing value 
    df_all['datetime'] = pd.to_datetime(df_all['date'])
    df_all.drop('date',axis=1, inplace=True)
    df_all = df_all.sort_values('datetime')
    df_all = df_all.drop_duplicates('datetime', ignore_index=True)
    
    # save file
    df_all.to_csv(filename,index=False)

bkk 13
chiang-mai 36
chiang-rai 25
kungming 24
luang-prabang 24
sikhottabong 39
tada_u 24
tak 26
yangong 29


In [89]:
# find missing date for chiang-mai data 
df_all = pd.read_csv('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities/chiang-mai.csv')
df_all['datetime'] = pd.to_datetime(df_all['datetime'] )
# find exisiting date 
ex_date = df_all['datetime'].dt.strftime('%Y-%m-%d').unique()
ex_date = set(ex_date)
# calculate the datelist 
start_date = datetime(2000, 10, 1)
stop_date = datetime.now()
date_range = pd.date_range(start_date, stop_date).strftime('%Y-%m-%d')
missing_date = list(set(date_range).difference(ex_date))

In [87]:
with open('../data/weather_cities/weather_station_info.json','r') as f:
    station_dict_list = json.load(f)
   
i = 0 
city_json = station_dict_list[i]
print(city_json)

{'city_name': 'Mueang Chiang Mai', 'province': 'Chiang Mai', 'country': 'Thailand', 'station_name': 'Chiang Mai International Airport Station', 'specific_url': 'th/mueang-chiang-mai/', 'latitude': '18.8 °N', 'longitude': '98.97 °E'}


In [90]:
 bad_date_df = scrape_weather(city_json, date_range=missing_date, data_folder='C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities/chiang-mai/')

C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities/chiang-mai/mueang-chiang-mai_weather.csv 2000-10-23


..\src\data\dl_weather.py:20: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 20 of the file ..\src\data\dl_weather.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(innerhtml)


In [97]:
# weather data 
filename = 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities/chiang-mai.csv'
wea = pd.read_csv(filename)
wea['datetime']  = pd.to_datetime(wea['datetime'])
# roud datetiem to whole 30 mins 
wea['datetime'] = wea['datetime'].dt.round('30T')

dates = wea['datetime'].dropna().dt.date.unique()

# fill in the missing value
new_datetime = pd.date_range(start=dates[0], end=dates[-1], freq='30T') 
new_weather = pd.DataFrame(new_datetime, columns=['datetime'])
new_weather = new_weather.merge(wea, on='datetime',how='left')
print(new_weather.shape)

# remove strange T reading
lowest_t = 5 
idx = new_weather[new_weather['Temperature(C)']< lowest_t].index
new_weather.loc[idx,['Temperature(C)','Dew Point(C)','Humidity(%)']] = np.nan

highest_t = 60
idx = new_weather[new_weather['Temperature(C)']> highest_t].index
new_weather.loc[idx,['Temperature(C)','Dew Point(C)','Humidity(%)']] = np.nan

new_weather = new_weather.fillna(method='ffill',limit=12)
new_weather = new_weather.fillna(method='bfill',limit=12)
new_weather = new_weather.set_index('datetime')
new_weather = new_weather.dropna(how='all').reset_index()
new_weather.to_csv(filename,index=False)

(333126, 11)


## Assemble from raw data 

In [7]:
# weather folder
w_folder = '../data/weather_cities/'

In [8]:
# extract station information
city_names = ['Mueang Chiang Mai', 'Mueang Chiang Rai', 'Mueang Tak','Bangkok','Yangon', 'Tada-U', 'Sikhottabong', 'Luang Prabang District','Kunming']
weather_station_info = find_weather_stations(city_names, weather_json_file=w_folder+'weather_station_info.json')
len(weather_station_info)

9

In [11]:
# concatenate raw data files
for city_json in weather_station_info: 
    # read existing file 
    city_name = ('_').join(city_json['city_name'].split(' '))
    current_filename = w_folder + city_name + '.csv'
    print(current_filename)
    
    # locate file in the city folder
    files = glob(w_folder + city_name + '/*.csv')
    print(len(files))
    
    weather_all = pd.DataFrame()
    for file in files:
        try:
            df = pd.read_csv(file)
        except:
            pass
        else:
            if 'date'== df.columns[-1]:
                df.columns = df.columns.str.replace('date','datetime')
            weather_all = pd.concat([weather_all, df],ignore_index=True)
        
    print(weather_all.columns)
    weather_all.to_csv(current_filename, index=False)
    

../data/weather_cities/Mueang_Chiang_Mai.csv
36
Index(['Time', 'Temperature(C)', 'Dew Point(C)', 'Humidity(%)', 'Wind',
       'Wind Speed(mph)', 'Wind Gust(mph)', 'Pressure(in)', 'Precip.(in)',
       'Condition', 'datetime'],
      dtype='object')
../data/weather_cities/Mueang_Chiang_Rai.csv
25
Index(['Time', 'Temperature(C)', 'Dew Point(C)', 'Humidity(%)', 'Wind',
       'Wind Speed(mph)', 'Wind Gust(mph)', 'Pressure(in)', 'Precip.(in)',
       'Condition', 'datetime'],
      dtype='object')
../data/weather_cities/Mueang_Tak.csv
26
Index(['Time', 'Temperature(C)', 'Dew Point(C)', 'Humidity(%)', 'Wind',
       'Wind Speed(mph)', 'Wind Gust(mph)', 'Pressure(in)', 'Precip.(in)',
       'Condition', 'datetime'],
      dtype='object')
../data/weather_cities/Bangkok.csv
13
Index(['Time', 'Temperature(C)', 'Dew Point(C)', 'Humidity(%)', 'Wind',
       'Wind Speed(mph)', 'Wind Gust(mph)', 'Pressure(in)', 'Precip.(in)',
       'Condition', 'datetime', 'Temperature(F)', 'Dew Point(F)',
      

In [12]:
# fix the unit of windspeed 
files = glob(w_folder + '/*.csv')
for file in files:
    print(file)
    df = pd.read_csv(file)
    df[['Wind Speed(mph)', 'Wind Gust(mph)']] = df[['Wind Speed(mph)', 'Wind Gust(mph)']]*1.60934
    df[['Wind Speed(mph)', 'Wind Gust(mph)']] = df[['Wind Speed(mph)', 'Wind Gust(mph)']].round(0)
    df.columns = df.columns.str.replace('mph', 'kmph')
    df.to_csv(file,index=False)

../data/weather_cities\Bangkok.csv
../data/weather_cities\Kunming.csv
../data/weather_cities\Luang_Prabang_District.csv
../data/weather_cities\Mueang_Chiang_Mai.csv
../data/weather_cities\Mueang_Chiang_Rai.csv
../data/weather_cities\Mueang_Tak.csv
../data/weather_cities\Sikhottabong.csv
../data/weather_cities\Tada-U.csv
../data/weather_cities\Yangon.csv


In [188]:
# drop duplicate datetime 
files = glob(w_folder + '/*.csv')
for file in files:
    df = pd.read_csv(file)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.sort_values('datetime')
    df = df.drop_duplicates('datetime')
    df.to_csv(file,index=False)

In [17]:
# Fill the missing value in raw Weather Data 
for file in files:
    df = pd.read_csv(file)
    df = fill_missing_weather(df,limit=12)
    df.to_csv(file, index=False)

## Update weather all cities

In [240]:
i = 0
start_date = datetime(2020, 1, 1)
end_date = datetime.now()
city_json = weather_station_info[i]
# read existing file 
city_name = ('_').join(city_json['city_name'].split(' '))
current_filename = w_folder + city_name + '.csv'
print(current_filename)

# obtain a list of existed dates
wea = pd.read_csv(current_file)
wea['datetime'] = pd.to_datetime(wea['datetime'])
# find exisiting date 
ex_date = wea['datetime'].dt.strftime('%Y-%m-%d').unique()
ex_date = set(ex_date)

# calculate the missing dates 
date_range = pd.date_range(start_date, end_date).strftime('%Y-%m-%d')
missing_date = list(set(date_range).difference(ex_date))
missing_date.sort()
len(missing_date)

../data/weather_cities/Mueang_Chiang_Mai.csv


15

In [18]:
# extract station information
city_names = ['Mueang Chiang Mai', 'Mueang Chiang Rai', 'Mueang Tak','Bangkok','Yangon', 'Tada-U', 'Sikhottabong', 'Luang Prabang District','Kunming']
weather_station_info = find_weather_stations(city_names, weather_json_file=w_folder+'weather_station_info.json')
len(weather_station_info)

9

In [19]:
for city_json in tqdm(weather_station_info):
    
    start_date = datetime(2000,10,1)
    end_date = datetime.now()
    update_weather(city_json, data_folder=w_folder, start_date=start_date, end_date=end_date)


  0%|                                                                                                               | 0/9 [00:00<?, ?it/s]

updateing file: ../data/weather_cities/Mueang_Chiang_Mai.csv
missing date 128




0it [00:00, ?it/s]

1it [00:28, 28.69s/it]

2it [01:02, 30.17s/it]

3it [01:35, 30.96s/it]

4it [02:04, 30.43s/it]

5it [02:33, 30.00s/it]

6it [03:02, 29.67s/it]

7it [03:30, 29.33s/it]

8it [04:00, 29.58s/it]

9it [04:29, 29.39s/it]

10it [04:59, 29.50s/it]

11it [05:32, 30.57s/it]

12it [06:05, 31.37s/it]

13it [06:35, 30.90s/it]

14it [07:04, 30.41s/it]

15it [07:34, 30.11s/it]

16it [08:03, 29.85s/it]

17it [08:37, 31.13s/it]

18it [09:07, 30.57s/it]

19it [09:39, 31.17s/it]

20it [10:11, 31.50s/it]

21it [10:40, 30.53s/it]

22it [11:08, 30.01s/it]

23it [11:42, 31.02s/it]

24it [12:14, 31.32s/it]

25it [12:42, 30.44s/it]

26it [13:10, 29.78s/it]

27it [13:42, 30.24s/it]

28it [14:10, 29.67s/it]

29it [14:42, 30.39s/it]

30it [15:11, 29.88s/it]

31it [15:40, 29.61s/it]

32it [16:09, 29.38s/it]

33it [16:40, 29.98s/it]

34it [17:09, 29.79s/it]

35it [17:39, 29.60s/it]

36it [18:09, 29.97s/it]

37it [18:41, 30.47s/it]

38it [19:14, 31.08s/it]

39it [19:47, 31.71s/it]

40it [20:16,

updateing file: ../data/weather_cities/Mueang_Chiang_Rai.csv
missing date 123




0it [00:00, ?it/s]

1it [00:34, 34.03s/it]

2it [01:07, 33.85s/it]

3it [01:36, 32.30s/it]

4it [02:04, 31.13s/it]

5it [02:37, 31.73s/it]

6it [03:10, 32.17s/it]

7it [03:39, 31.06s/it]

8it [04:07, 30.29s/it]

9it [04:37, 30.01s/it]

10it [05:05, 29.61s/it]

11it [05:38, 30.47s/it]

12it [06:06, 29.78s/it]

13it [06:34, 29.27s/it]

14it [07:02, 28.98s/it]

15it [07:31, 28.83s/it]

16it [08:00, 28.99s/it]

17it [08:29, 28.96s/it]

18it [08:58, 28.87s/it]

19it [09:26, 28.58s/it]

20it [09:54, 28.50s/it]

21it [10:22, 28.49s/it]

22it [10:53, 28.96s/it]

23it [11:22, 29.03s/it]

24it [11:51, 29.07s/it]

25it [12:20, 29.01s/it]

26it [12:50, 29.39s/it]

27it [13:24, 30.70s/it]

28it [13:52, 29.98s/it]

29it [14:20, 29.50s/it]

30it [14:49, 29.31s/it]

31it [15:19, 29.29s/it]

32it [15:50, 29.91s/it]

33it [16:23, 30.76s/it]

34it [16:52, 30.23s/it]

35it [17:21, 29.96s/it]

36it [17:49, 29.44s/it]

37it [18:18, 29.30s/it]

38it [18:49, 29.61s/it]

39it [19:17, 29.35s/it]

40it [19:48,

...



0it [00:00, ?it/s]

1it [00:55, 55.16s/it]

2it [01:33, 50.22s/it]

3it [02:05, 44.68s/it]

4it [02:44, 43.06s/it]

5it [03:22, 41.48s/it]

6it [03:55, 38.93s/it]

7it [04:26, 36.37s/it]

8it [04:56, 34.47s/it]

9it [05:26, 33.14s/it]

10it [05:56, 32.23s/it]

11it [06:32, 33.51s/it]

12it [07:03, 32.58s/it]

13it [07:36, 32.75s/it]

14it [08:09, 32.85s/it]

15it [08:40, 32.36s/it]

16it [09:10, 31.48s/it]

17it [09:39, 30.95s/it]

18it [10:10, 30.82s/it]

19it [10:39, 30.45s/it]

20it [11:09, 30.22s/it]

21it [11:40, 30.43s/it]

22it [12:12, 30.87s/it]

23it [12:42, 30.70s/it]

24it [13:12, 30.55s/it]

25it [13:45, 31.15s/it]

26it [14:15, 30.89s/it]

27it [14:44, 30.38s/it]

28it [15:17, 30.98s/it]

29it [15:50, 31.58s/it]

30it [16:19, 30.83s/it]

31it [16:52, 31.58s/it]

32it [17:26, 32.12s/it]

33it [17:59, 32.54s/it]

34it [18:32, 32.74s/it]

35it [19:02, 31.82s/it]

36it [19:32, 31.19s/it]

37it [20:01, 30.63s/it]

38it [20:37, 32.27s/it]

39it [21:11, 32.90s/it]

40it [21:41,

updateing file: ../data/weather_cities/Luang_Prabang_District.csv
missing date 129




0it [00:00, ?it/s]

1it [00:49, 49.16s/it]

2it [01:29, 46.52s/it]

3it [02:00, 41.96s/it]

4it [02:37, 40.51s/it]

5it [03:09, 37.94s/it]

6it [03:46, 37.45s/it]

7it [04:22, 37.14s/it]

8it [04:59, 36.96s/it]

9it [05:35, 36.78s/it]

10it [06:10, 36.15s/it]

11it [06:45, 35.90s/it]

12it [07:17, 34.62s/it]

13it [07:51, 34.66s/it]

14it [08:25, 34.46s/it]

15it [09:01, 34.92s/it]

16it [09:36, 34.82s/it]

17it [10:07, 33.64s/it]

18it [10:45, 34.93s/it]

19it [11:24, 36.28s/it]

20it [12:02, 36.77s/it]

21it [12:42, 37.69s/it]

22it [13:17, 37.02s/it]

23it [13:51, 36.10s/it]

24it [15:24, 53.19s/it]

25it [16:01, 48.13s/it]

26it [16:35, 43.89s/it]

27it [17:08, 40.77s/it]

28it [17:55, 42.42s/it]

29it [18:29, 39.99s/it]

30it [19:04, 38.65s/it]

31it [19:41, 38.16s/it]

32it [20:18, 37.70s/it]

33it [20:52, 36.58s/it]

34it [21:28, 36.46s/it]

35it [22:02, 35.76s/it]

36it [22:37, 35.34s/it]

37it [23:13, 35.71s/it]

38it [23:51, 36.23s/it]

39it [24:25, 35.78s/it]

40it [24:56,

updateing file: ../data/weather_cities/Kunming.csv
missing date 129




0it [00:00, ?it/s]

1it [00:30, 30.91s/it]

2it [01:03, 31.32s/it]

3it [01:30, 30.24s/it]

4it [02:05, 31.62s/it]

5it [02:37, 31.53s/it]

6it [03:05, 30.70s/it]

7it [03:34, 30.06s/it]

8it [04:06, 30.79s/it]

9it [04:38, 30.89s/it]

10it [05:10, 31.38s/it]

11it [05:42, 31.59s/it]

12it [06:17, 32.64s/it]

13it [06:46, 31.34s/it]

14it [07:17, 31.49s/it]

15it [07:49, 31.53s/it]

16it [08:21, 31.54s/it]

17it [08:52, 31.61s/it]

18it [09:25, 31.83s/it]

19it [09:57, 32.04s/it]

20it [10:26, 30.92s/it]

21it [10:57, 31.13s/it]

22it [11:25, 30.20s/it]

23it [11:57, 30.55s/it]

24it [12:25, 29.93s/it]

25it [12:53, 29.44s/it]

26it [13:22, 29.09s/it]

27it [13:49, 28.64s/it]

28it [14:17, 28.43s/it]

29it [14:48, 29.21s/it]

30it [15:17, 29.06s/it]

31it [15:44, 28.62s/it]

32it [16:12, 28.41s/it]

33it [16:41, 28.38s/it]

34it [17:08, 28.14s/it]

35it [17:40, 29.16s/it]

36it [18:11, 29.77s/it]

37it [18:43, 30.35s/it]

38it [19:15, 30.92s/it]

39it [19:43, 30.18s/it]

40it [20:15,

# Holiday In Thailand

https://www.timeanddate.com/holidays/thailand/2001

In [13]:
years = np.arange(2000,2021)

In [14]:
# scrape holiday from all websites
th_holiday = pd.DataFrame()
for year in years:
    url = f'https://www.timeanddate.com/holidays/thailand/{year}'
    df = pd.read_html(url)[0]
    df['year'] = year
    th_holiday = pd.concat([th_holiday,df],ignore_index=True)

In [17]:
th_holiday.columns = ['Date', 'day_of_week','name','type','year']

In [22]:
th_holiday = th_holiday[~th_holiday['Date'].isna()]

In [28]:
th_holiday['date'] = th_holiday['Date'] + ', ' + th_holiday['year'].astype(str)
th_holiday['date'] = pd.to_datetime(th_holiday['date'])

In [30]:
th_holiday.to_csv('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/th_holiday.csv',index=False)

# Hotspots Data 

In [35]:
# load stations information for Chiangmai
station_info_file = aqm_folder + 'stations_locations.json'
with open(station_info_file, 'r',encoding="utf8") as f:
    station_info = json.load(f)
station_info = station_info['stations']

print(station_info[124])

{'stationID': '36t', 'nameTH': 'โรงเรียนยุพราชวิทยาลัย ', 'nameEN': 'Yupparaj Wittayalai School', 'areaTH': 'ต.ศรีภูมิ อ.เมือง, เชียงใหม่', 'areaEN': 'Si Phum, Meuang, Chiang Mai', 'stationType': 'GROUND', 'lat': '18.7909205', 'long': '98.9881062', 'LastUpdate': {'date': '2020-03-27', 'time': '03:00', 'PM25': {'value': '87', 'unit': 'µg/m³'}, 'PM10': {'value': '110', 'unit': 'µg/m³'}, 'O3': {'value': 'N/A', 'unit': 'ppb'}, 'CO': {'value': '1.09', 'unit': 'ppm'}, 'NO2': {'value': '-', 'unit': 'ppb'}, 'SO2': {'value': '2', 'unit': 'ppb'}, 'AQI': {'Level': '4', 'aqi': '192'}}}


In [36]:
# obtain the lat and long in km
lat_km = (merc_y(station_info[124]['lat'])/1E3 ).round()
long_km = (merc_x(station_info[124]['long'])/1E3).round()
print(lat_km,  long_km )

2117.0 11019.0


In [37]:
# file map folder 
m_files = glob('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/fire_map/world_2000-2020/M6/*.csv')
v_files = glob('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/fire_map/world_2000-2020/V1/*.csv')

In [38]:
# keep the file spot with distance 1000 km from the station
distance = 1000 # km

filename = 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/cm_proc/file_m.csv'


file_all = pd.DataFrame()

for file in tqdm(m_files):
    f = pd.read_csv(file)
     
    # convert lat 
    f['lat_km'] = (f['latitude'].apply(merc_y)/1E3).round().astype(int)
    f['long_km'] = (merc_x(f['longitude'])/1E3).round().astype(int)
    # remove by lat 
    f = f[f['lat_km'] <= (lat_km+1000)]
    f = f[f['lat_km'] >= (lat_km-1000)]
    # remove by long 

    f = f[f['long_km'] <= (long_km+1000)]
    f = f[f['long_km'] >= (long_km-1000)]
     
    file_all = pd.concat([file_all,f],ignore_index=True)
        
        
file_all.to_csv(filename, index=False)

In [39]:
m_filename = 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/cm_proc/file_m.csv'
process_fire_data(m_filename)


c:\users\benny\pyenv\geo\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(3901918, 17)


In [40]:
# keep the file spot with distance 1000 km from the station
distance = 1000 # km

filename = 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/cm_proc/file_v.csv'

fire_all = pd.DataFrame()

for file in v_files:
    f = pd.read_csv(file)
     
    # convert lat 
    f['lat_km'] = (f['latitude'].apply(merc_y)/1E3).round()
    f['long_km'] = (merc_x(f['longitude'])/1E3).round()
    # remove by lat 
    f = f[f['lat_km'] <= (lat_km+1000)]
    f = f[f['lat_km'] >= (lat_km-1000)]
    # remove by long 

    f = f[f['long_km'] <= (long_km+1000)]
    f = f[f['long_km'] >= (long_km-1000)]
    
    fire_all = pd.concat([fire_all,f],ignore_index=True)  
     
        
fire_all.to_csv(filename, index=False)

In [41]:
df = pd.read_csv('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/cm_proc/file_v.csv')
df.shape

c:\users\benny\pyenv\geo\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (10,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(9092222, 17)